## Deep learning quantum DD compiler

## Methodology

1. **Model training.** The deep learning model was trained on 1,000 random 5-qubit circuits evaluated by IBMQ Burlington to predict the difference in trace distance between two equivalent random circuits. These supremacy-style random circuits had many gaps due to 2-qubit gates compensating for partial connectivity after compiling to maximum optimization; we filled the gaps completely randomly with sequences of U3 gates equivalent to the identity. All 1,000 circuits were run on IBM Q Burlington.

2. **Compiling.** We ran the trained model on a new set of 500 random circuits. For each random circuit (compiled to maximum optimization for IBM Q Burlington's error rates on March 18), we generated 1,000 randomly padded circuits. The deep learning model was then used to predict the relative noise between pairs of circuits in 50 tournaments of 20 competitors each, selecting the circuit expected to be the least noisy as the "compiled" circuit.

3. **Testing.** The compiled circuits were run on IBM Q Burlington, Essex, London, Ourense, Vigo (which have the same architecture) and Yorktown (which has a different architecture). On Burlington, we found that DD-compiled circuits improve trace distance by 11% (10% to 12%, 0.95 CI) per circuit compared to IBM-compiled. On all other 5-qubit devices, the DD compiler also performed better than the IBM Qiskit compiler, although with around half the improvement compared to performance on Burlington. Nevertheless, the improvement on all devices was statistically significantly better than applying random DD sequences.

In [2]:
import numpy as np
import bootstrapped.bootstrap as bs
import bootstrapped.stats_functions as bs_stats
import glob

Average impact of random DD sequences on the training set:

In [14]:
a = np.load('supremacy_all_5_unique/burlington_noise.npy')
free = np.expand_dims(a[:, 0], 1)
compiled = a[:, 1:]
bs.bootstrap(((compiled - free)/free).flatten(), stat_func=bs_stats.mean)

-0.04472841360952699    (-0.04586115666298597, -0.043587626767780586)

We evaluate the change in trace distance between compiled and identity for all computers, as well as the difference in trace distance between a given computer and the Burlington device for which the noise model was trained for.

In [5]:
def diff(files):
    data = []
    for i in range(len(files)):
        d = []
        for f in sorted(files[i]):
            d.append(np.load(f))
        if len(d) == 0:
            return None
        data.append(np.concatenate(d))
    diff = (data[0][:len(data[1])] - data[1])/data[1]
    return diff

In [15]:
home = 'test_noise_5/'
home_files = [glob.glob(home + 'burlington_compiled*'), glob.glob(home + 'burlington_identity*')]
home_diff = diff(home_files)

print('Computer: burlington')
print('burlington change in trace distance', bs.bootstrap(home_diff, stat_func=bs_stats.mean))
print(len(home_diff), 'circuits')
print()

comparisons = ['essex', 'london', 'ourense', 'vigo', '5_yorktown']
for c in comparisons:
    files = [glob.glob(home + c + '_compiled*'), glob.glob(home + c + '_identity*')]
    dr_diff = diff(files)
    print('Computer:', c)
    if dr_diff is None:
        print('no data')
    else:
        l = min(len(dr_diff), len(home_diff))
        print(c + ' change in trace distance', bs.bootstrap(dr_diff, stat_func=bs_stats.median))
        print('diff from burlington', bs.bootstrap(dr_diff[:l] - home_diff[:l], stat_func=bs_stats.median))
        print(l, 'circuits')
    print()

Computer: burlington
burlington change in trace distance -0.11140341171900789    (-0.12448761411948694, -0.09776461871487682)
500 circuits

Computer: essex
essex change in trace distance -0.05544860532039153    (-0.07066875273166959, -0.0426917043367816)
diff from burlington 0.0495800995168085    (0.0386419387018658, 0.06341252505688803)
500 circuits

Computer: london
london change in trace distance -0.05561113504815794    (-0.0594903253374298, -0.04926013584502502)
diff from burlington 0.04336468345014906    (0.027821642077278205, 0.05423607741455928)
500 circuits

Computer: ourense
ourense change in trace distance -0.06148141545139791    (-0.07073663543558335, -0.05315337939908586)
diff from burlington 0.03574368774382461    (0.02318768360675203, 0.04752435282444325)
500 circuits

Computer: vigo
vigo change in trace distance -0.047762428196207886    (-0.05629225779499844, -0.037619481706219135)
diff from burlington 0.04772189166971301    (0.02519764654574376, 0.0654621371629196)
500 